In [1]:
import win32com.client
import pandas as pd
import numpy as np
import datetime as dt
import time
from tqdm import tqdm

In [2]:
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
instStockChart = win32com.client.Dispatch('CpSysDib.CpSvr7254')

In [3]:
# 대신증권 API 연결 확인. 1 출력시 연결 성공, 0 출력시 연결 실패
def connectCheck():
    print('[시스템]:증권사 연결 시도 중')
    if(instCpCybos.IsConnect == 1):
        print('[시스템]:증권사 연결 완료!')

In [4]:
# 증권시장에서 상장되어 있는 종목의 갯수 출력 -> 새로운 종목이 상장되거나 폐지됨에 따라 값이 달라진다.
def canTradeStockCount():
    print('[시스템]:거래 가능한 종목 개수 : '+str(instCpStockCode.GetCount())+'개 체크')

## 데이터 수집

In [5]:
# 대신증권 기준으로 정렬된 상장된(주식거래가 가능한) 종목을 전부 가져오기
def getStockNameCode():
    print('[시스템]:거래 가능한 종목 리스트 목록 가져오는 중')
    code = [] # 종목코드
    name = [] # 종목명

    for i in range(0, instCpStockCode.GetCount()):
        code.append(instCpStockCode.GetData(0, i))
        name.append(instCpStockCode.GetData(1, i))

    codeNameList = pd.DataFrame({
        '종목코드':code
        , '종목명':name
    })
    # codeNameList
    if(len(codeNameList) == 0):
        print('[시스템]:거래 가능한 종목 리스트 목록 가져온 개수 : '+str(len(codeNameList))+'개')
        print('[시스템]:거래 가능한 종목 리스트 목록 가져오기에 문제가 발생하였습니다.')
    else:
        print('[시스템]:거래 가능한 종목 리스트 목록 가져오기 완료!')

    return codeNameList

In [6]:
def jongmokSerach(jongmokSearchList, codeNameList):
    code = []
    name = []
    for names in jongmokSearchList:
        code.append(codeNameList[codeNameList['종목명'] == names]['종목코드'].iloc[0])
        name.append(codeNameList[codeNameList['종목명'] == names]['종목명'].iloc[0])

    selCodeNameList = pd.DataFrame({
        '종목코드':code
        , '종목명':name
    })
    return selCodeNameList

In [7]:
def getjusik(jusikCode, jusikName, start, end):
    pdListed = []

    instStockChart.SetInputValue(0, jusikCode)
    instStockChart.SetInputValue(1, 0)
    instStockChart.SetInputValue(2, start)
    instStockChart.SetInputValue(3, end)
    instStockChart.SetInputValue(4, ord('0'))
    instStockChart.SetInputValue(5, 0)
    instStockChart.SetInputValue(6, ord('1'))

    instStockChart.BlockRequest()

    num = instStockChart.GetHeaderValue(1)
                                                            
    for i in range(num):
        Listed = {}
        Listed['종목코드'] = jusikCode                             # 종목코드
        Listed['종목명'] = jusikName                               # 종목명
        Listed['날짜'] = instStockChart.GetDataValue(0, i)        # 날짜
        Listed['개인'] = instStockChart.GetDataValue(1, i)        # 개인
        Listed['외국인'] = instStockChart.GetDataValue(2, i)      # 외국인
        Listed['기관계'] = instStockChart.GetDataValue(3, i)      # 기관계
        Listed['금융투자'] = instStockChart.GetDataValue(4, i)    # 금융투자
        Listed['보험'] = instStockChart.GetDataValue(5, i)        # 보험
        Listed['투신'] = instStockChart.GetDataValue(6, i)        # 투신
        Listed['은행'] = instStockChart.GetDataValue(7, i)        # 은행
        Listed['기타금융'] = instStockChart.GetDataValue(8, i)    # 기타금융
        Listed['연기금'] = instStockChart.GetDataValue(9, i)      # 연기금
        Listed['기타법인'] = instStockChart.GetDataValue(10, i)   # 기타법인
        Listed['기타외인'] = instStockChart.GetDataValue(11, i)   # 기타외인
        Listed['사모펀드'] = instStockChart.GetDataValue(12, i)   # 사모펀드

        pdListed.append(Listed)

    dataDf = pd.DataFrame(pdListed)
    return dataDf

In [8]:
def aaa(stockNameList):
    return pd.read_csv('../주식데이터 수집/'+stockNameList[0]+' 주식 데이터(투자주체별).csv', encoding='euc-kr')

In [9]:
# 수신 받은 주식데이터(투자주체별) 정렬하기
def reverse(stockNameList):
    print('[시스템]:수신받은 데이터 날짜, 시간순으로 정렬 작업중')
    for i in range(len(stockNameList)):
        data1 = pd.read_csv('./stockdata/'+stockNameList[i]+' 주식 데이터(투자주체별).csv', encoding='euc-kr')

        data1.sort_values(['날짜'], inplace=True)
        data1.reset_index(inplace=True)
        data1.drop('index', axis=1, inplace=True)
        data1.to_csv('../주식데이터 수집/'+stockNameList[i]+' 주식 데이터(투자주체별).csv', encoding='euc-kr', index=False)
    print('[시스템]:정렬 완료!')

In [10]:
stockNameList = ['셀바스AI'] # 이곳에 수신 받고 싶은 종목 이름을 정확히 입력해주세요. 여러개 가능.

connectCheck()
canTradeStockCount()
codeNameList = getStockNameCode()
selCodeNameList = jongmokSerach(stockNameList, codeNameList)

a = dt.datetime(2021,1,1) # 수신 받고 싶은 원하는 시작 날짜를 입력하세요.
b = dt.datetime(2023,7,19) # 수신 받고 싶은 원하는 종료 날짜를 입력하세요.
number = b-a

for j in range(0, selCodeNameList.shape[0]):
    print('[시스템]:'+str(a.strftime('%Y%m%d'))+' ~ '+str(b.strftime('%Y%m%d'))+' 사이에 거래된 '+str(selCodeNameList['종목명'].iloc[j])+' 종목 주가 데이터(투자주체별) 가져오는 중')
    frame = []
    for i in tqdm(range(0, number.days, 2)):
        c = a + dt.timedelta(days=i)
        d = a + dt.timedelta(days=i+1)
        result = getjusik(selCodeNameList['종목코드'].iloc[j], selCodeNameList['종목명'].iloc[j], c.strftime('%Y%m%d'), d.strftime('%Y%m%d'))
        frame.append(result)
        time.sleep(0.3)
    hapdata = pd.concat(frame)
    hapdata.to_csv('../주식데이터 수집/'+selCodeNameList['종목명'].iloc[j]+' 주식 데이터(투자주체별).csv', encoding='euc-kr', index=False)
    print('[시스템]:'+selCodeNameList['종목명'].iloc[j]+' 종목의 주식 데이터(투자주체별) 요청 수신성공 및 csv로 저장 성공!')

reverse(stockNameList)
print('[시스템]:수신요청 종료')
aaa(stockNameList)

[시스템]:증권사 연결 시도 중
[시스템]:증권사 연결 완료!
[시스템]:거래 가능한 종목 개수 : 3996개 체크
[시스템]:거래 가능한 종목 리스트 목록 가져오는 중


[시스템]:거래 가능한 종목 리스트 목록 가져오기 완료!
[시스템]:20210101 ~ 20230719 사이에 거래된 셀바스AI 종목 주가 데이터(투자주체별) 가져오는 중


100%|██████████| 465/465 [02:26<00:00,  3.18it/s]


[시스템]:셀바스AI 종목의 주식 데이터(투자주체별) 요청 수신성공 및 csv로 저장 성공!
[시스템]:수신받은 데이터 날짜, 시간순으로 정렬 작업중


FileNotFoundError: [Errno 2] No such file or directory: './stockdata/셀바스AI 주식 데이터(투자주체별).csv'